In [51]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

symbol = "SPY"
base_folder = r"C:\Users\MAYANK DANGWAL\.zipline\csv\yahoo\daily"
# download dataframe using pandas_datareader
df = pdr.get_data_yahoo(symbol, start="2023-01-01", end="2023-12-10")
df = df.reset_index()
df = df.drop(columns="Close").rename(columns={"Adj Close": "Close"})
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df['dividend'] = 0.0
df['split'] = 1.0
df['date'] = pd.to_datetime(df.date)
print(df.head(2))
file_path = "{}\{}.csv".format(base_folder,symbol)
df.to_csv(file_path, index=False)






[*********************100%%**********************]  1 of 1 completed
        date        open        high         low       close    volume  \
0 2023-01-03  384.369995  386.429993  377.829987  375.118713  74850700   
1 2023-01-04  383.179993  385.880005  380.000000  378.014740  85934100   

   dividend  split  
0       0.0    1.0  
1       0.0    1.0  


In [94]:
from zipline.data.bundles import _bundle_registry
print(_bundle_registry.get('equity-bundle', None))

ImportError: cannot import name '_bundle_registry' from 'zipline.data.bundles' (C:\Users\pthon\anaconda3\envs\ta_env\lib\site-packages\zipline\data\bundles\__init__.py)

In [93]:
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.run_pipeline_against_bundle import run_pipeline_against_bundle
run_pipeline_against_bundle(Pipeline({'close': USEquityPricing.close.latest}),'2017','2018',bundle='equities-bundle')

UnknownBundle: No bundle registered with the name 'equities-bundle'

In [79]:
%load_ext zipline

In [97]:

%%zipline --start 2023-01-01 --end 2023-12-10 --output single_factor.pickle --no-benchmark --bundle equity-bundle
#start="2023-01-01", end="2023-12-10"
from zipline.api import (
    attach_pipeline,
    date_rules,
    time_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    get_open_orders,
    calendars
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
import numpy as np
import pandas as pd

MONTH = 21
YEAR = 12 * MONTH
N_LONGS = N_SHORTS = 25
VOL_SCREEN = 1000

class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())


def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs': mean_reversion.bottom(N_LONGS),
                             'shorts': mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))


def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)


def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, 'price'))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = set(context.portfolio.positions.keys()) - set(longs.union(shorts))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS)


def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)
    context.set_commission(commission.PerShare(cost=.01, min_trade_cost=0))
    context.set_slippage(slippage.VolumeShareSlippage())


def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')

UnknownBundle: No bundle registered with the name 'equity-bundle'